# Imports

In [1]:
# Machine learning
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from abc import abstractmethod
from tensorflow.keras import layers


# Python library
import math
import statistics

# Custom imports
from tensorflow.python.framework import ops

In [2]:
# Where to find the files that I save
from paths import *

from utilities import *
init_plt()
init_tf()
tf.keras.backend.set_floatx('float32')

# Parameters

In [3]:
# All parameters dict
NUMBER_OF_FEATURES = 50 #9
p = {
    "experiment_type": "predicting_fhat",
    "experiment_name": "6_18_dropout_1_layer",
    "k": 5,
    "select_young": False,
    "select_old": False,
    "override_weights_to_one": False,
    "eps": 0.0,
    "enable_one_hot": True,
    
    "models_layers_and_parameters": {
        "weights": {
                "input_size": NUMBER_OF_FEATURES, # Do not change
                "layers":[
                    # Format: (Size, Activation Function (None if none))
                    # Last (output) layer should have size 1
                    #(128, tf.nn.sigmoid),
                    #(64, tf.nn.sigmoid),
                    (32, tf.nn.sigmoid),
                    (32, tf.nn.sigmoid),
                    (1, None)
                ],
                "learning_rate": 1e-3,
                "training_steps": 5000,
                "dropout": 0.2,
                "display_step": 1000,

        },
    }
}


assert(not (p["select_young"] and p["select_old"])) # Make sure both are not selected
OUTPUT_FOLDER = GENERAL_RESULTS_FOLDER + p["experiment_name"] + SLASH
DISPLAY_STEP = 500

In [4]:
base_model_parameters = {
    "input_size": NUMBER_OF_FEATURES,
    "learning_rate": 1e-3,
    "training_steps": 5000,
    "display_step": 1000,
}
                

def make_fhat_neural_network_parameters(layer_size, dropout, resample_data, resample_data_points):
    d = dict(base_model_parameters)
    d.update({
        "layers": [
            # Format: (Size, Activation Function (None if none))
            # Last (output) layer should have size 1
            #(128, tf.nn.sigmoid),
            #(64, tf.nn.sigmoid),
            (layer_size, tf.nn.sigmoid),
            #(layer_size, tf.nn.sigmoid),
            #(layer_size, tf.nn.sigmoid),
            (4, None)
        ],
        "dropout": dropout,
        "resample_data": resample_data,
        "resampled_points_per_category": resample_data_points,
    })
    return d


possible_layer_sizes = [ 
    #2, 
    #4, 
    #8, 
    #16, 
    32, 
    #64, 
    #128, 
]

possible_dropout = [
    None,
    0.02,
    0.04,
    0.06,
    0.08,
    0.10,
    0.12,
    0.14,
    0.16,
    0.18,
    0.20,
    0.22,
    0.24,
    0.26,
    0.28,
    0.30,
    0.32,
    0.34,
    0.36,
    0.38,
    0.40,
    0.42,
    0.44,
    0.46,
    0.48,
    0.64,
]

possible_resampling = [
    #(True, 10000),
    (False, None), # if False, second can be anything, it does not matter
    
]


all_model_parameters = [
    make_fhat_neural_network_parameters(layer_size, dropout, resample_data, resample_data_points)
    for layer_size in possible_layer_sizes
    for dropout in possible_dropout
    for (resample_data, resample_data_points) in possible_resampling
]

# Make the folder where the results will be placed in

In [5]:
! mkdir $OUTPUT_FOLDER

A subdirectory or file results\6_18_dropout_1_layer\ already exists.


# Data preparation

In [6]:
# Load entire data file
data = np.loadtxt(DATA_PATH).astype(np.float32)

# Shuffle data
np.random.shuffle(data)

young_mask = data[:, 0]==1
old_mask = data[:, 0]==2

# Select young or old
if p["select_young"]:
    print("We selected only young.")
    data = data[young_mask]
if p["select_old"]:
    print("We selected only old.")
    data = data[old_mask]

In [7]:
# Divide data into training, test, cross_validation
samples_per_fold = math.floor(data.shape[0]/p["k"])
samples = samples_per_fold*p["k"]
print("Samples:", samples)
bounds = list(map(lambda x: (x, x+samples_per_fold), list(range(0, samples_per_fold*p["k"], samples_per_fold))))

data = data[:p["k"]*samples_per_fold] # This is all we can use
young_mask = young_mask[:p["k"]*samples_per_fold]
old_mask = old_mask[:p["k"]*samples_per_fold]

Samples: 6330


In [8]:
# Extract features, y_1, y_2, weights

number_of_samples = data.shape[0]

if p["enable_one_hot"]:
    def make_one_hot(x):
        unique = list(set(x))
        convert_to_index = np.vectorize(lambda x: unique.index(x))

        one_hot = np.zeros((number_of_samples, len(unique)))
        one_hot[np.arange(number_of_samples), convert_to_index(x)] = 1

        one_hot = one_hot[:, :-1]
        return one_hot
else:
    def make_one_hot(x):
        return np.expand_dims(x, axis=1) # Do nothing, just add a dimension


def extract(data):

    age = (data[:, 0]==2).astype(np.float32)
    group = data[:, 21] # 
    hom = data[:,23]
    prof  = data[:, 43]; #
    reg = data[:, 46] #
    rena = data[:, 47]
    tracir = data[:, 52] #
    trage = data[:, 53] # 
    usag = data[:,56] # 
    zone = data[:, 57] #

    y_1 = (data[:, 9] > 0).astype(np.float32)
    y_2 = (data[:, 24] == 0).astype(np.float32)
    weights = data[:, 42]

    # Convert These variables to ones and zeros
    list_categorical_variables =         list(map(make_one_hot, [group, prof, reg, tracir, trage, usag, zone]))

    # Do not convert these (they only take values 0 and 1)
    add_dim = lambda x: np.expand_dims(x, axis=1)
    list_other_variables =         list(map(add_dim, [hom, rena, age]))


    # Just add here
    return np.concatenate(list_categorical_variables+list_other_variables, axis=1),            add_dim(y_1),            add_dim(y_2),            add_dim(weights)

X, y_1, y_2, weights = extract(data)
X = X.astype(np.float32)
y_1 = y_1.astype(np.float32)
y_2 = y_2.astype(np.float32)
print(X.shape)

(6330, 50)


In [9]:
# Normalize X

divide_by_std_dev = lambda x: x/np.std(x, axis=0, keepdims=True)
subtract_mean = lambda x: x-np.mean(x, axis=0, keepdims=True)

X = divide_by_std_dev(subtract_mean(X))

In [10]:
t = lambda x: np.squeeze(x, axis=1)

f00 = ((y_1 == 0) & (y_2 == 0)).astype(np.float32)
f01 = ((y_1 == 0) & (y_2 == 1)).astype(np.float32)
f10 = ((y_1 == 1) & (y_2 == 0)).astype(np.float32)
f11 = ((y_1 == 1) & (y_2 == 1)).astype(np.float32)

f = np.empty((y_1.shape[0], 4), dtype=np.float32)
f[:, 0] = t(f00)
f[:, 1] = t(f01)
f[:, 2] = t(f10)
f[:, 3] = t(f11)

# Training Helpers

In [11]:
from neural_network import create_neural_network
class NNModel():

    def __init__(self, model_layers_and_parameters):
        self.parameters = model_layers_and_parameters
        apply_network, variables, _ = create_neural_network(model_layers_and_parameters)
        self.variables = variables
        self.nn_layers = apply_network
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=model_layers_and_parameters["learning_rate"])

    @tf.function(experimental_relax_shapes=True)
    @abstractmethod # Means this function must be implemented
    def run(self, inp, training):
        pass 

    def train_one_step(self, inp):
        with tf.GradientTape() as tape:
            output = self.run(inp, training=True)

        loss = output["loss"]
        grads = tape.gradient(loss, self.variables)
        self.optimizer.apply_gradients(zip(grads, self.variables)) 

        return output

    def train(self, inp):

        for i in range(self.parameters["training_steps"]):
            output = self.train_one_step(inp)

            #if i % DISPLAY_STEP == 0:
                #print("Step", i, "loss is", output["loss"].numpy())

        return output

W0622 09:13:43.615142  7760 deprecation_wrapper.py:119] From D:\Sync\salanie\neural_network.py:3: The name tf.keras.initializers.he_normal is deprecated. Please use tf.compat.v1.keras.initializers.he_normal instead.

W0622 09:13:43.616835  7760 deprecation_wrapper.py:119] From D:\Sync\salanie\neural_network.py:5: The name tf.keras.initializers.RandomUniform is deprecated. Please use tf.compat.v1.keras.initializers.RandomUniform instead.

W0622 09:13:43.621203  7760 deprecation.py:506] From C:\Users\lilia\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
class WeightModel(NNModel):
    
    # This the run function we had to implement
    @tf.function(experimental_relax_shapes=True)
    def run(self, inp, training):
        output_unsigmoided, _ = self.nn_layers(inp["X"], training=training)
        output_sigmoided = tf.sigmoid(output_unsigmoided)
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=inp["weights"], logits=output_unsigmoided)
        loss = tf.reduce_mean(cross_entropy)
        return {
            "weights_predicted": output_sigmoided, 
            "loss": loss
        }
    
class FhatModel(NNModel):
    
    # This the run function we had to implement
    @tf.function(experimental_relax_shapes=True)
    def run(self, inp, training):
        
        unsoftmaxed, _ = self.nn_layers(inp["X"], training=training)
        softmaxed = tf.nn.softmax(unsoftmaxed, axis=1)
        softmaxed = softmaxed / tf.reduce_sum(softmaxed, axis=1, keepdims=True)
        weights_squeezed = tf.squeeze(inp["weights"], axis=1)
        
        
        entropied = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.math.argmax(inp["f"], axis=1), \
                                                                   logits=unsoftmaxed)
        loss = tf.reduce_mean(weights_squeezed*entropied)
        accuracy = 100.0*tf.reduce_mean(tf.cast(tf.math.equal(tf.math.argmax(softmaxed, axis=1), \
                                                              tf.math.argmax(inp["f"], axis=1)), tf.float32))
                
        f_list = tf.unstack(tf.expand_dims(softmaxed, axis=2), axis=1)
        fhat00 = f_list[0]
        fhat01 = f_list[1]
        fhat10 = f_list[2]
        fhat11 = f_list[3]
        
        return {
            "unsoftmaxed": unsoftmaxed,
            "softmaxed": softmaxed,
            "accuracy": accuracy,
            "loss": loss,
            "fhat00": fhat00, 
            "fhat01": fhat01, 
            "fhat10": fhat10, 
            "fhat11": fhat11,
        }

# Weight training

In [13]:
# Weights model training

# Remember the old ones
if "weights_original" not in locals().keys(): # If we haven't defined weights_original already
    weights_original = weights 

# Predict weights
if p["override_weights_to_one"]:
    weights = np.ones_like(weights)
else:
    weights = np.empty_like(weights)

    for k in range(p["k"]):

        lower, upper = bounds[k]
        get_kth = lambda x: x[lower:upper]
        cut_kth = lambda x: np.concatenate([x[:lower], x[upper:]], axis=0)
        X_kth, weights_kth = list(map(get_kth, [X, weights_original]))
        X_rest, weights_rest = list(map(cut_kth, [X, weights_original]))

        # Reset graph
        tf.keras.backend.clear_session()
        
        model = WeightModel(p["models_layers_and_parameters"]["weights"])
        
        inp = dict_to_tensors({"X": X_rest, "weights": weights_rest})
        _ = model.train(inp)
        
        inp = dict_to_tensors({"X": X_kth, "weights": weights_kth})
        output_kth = model.run(inp, training=False)
        print("Final loss on kth fold:", output_kth["loss"].numpy())
        weights[lower:upper] = output_kth["weights_predicted"].numpy() # Replace weights with predicted ones

W0622 09:13:44.103775  7760 deprecation.py:506] From C:\Users\lilia\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\autograph\impl\api.py:255: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0622 09:13:44.104772  7760 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0622 09:13:44.124690  7760 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0622 09:13:44.151617  7760 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0622 09:13:44.171589  7760 nn_ops.py:4224] Large dropout r

AttributeError: 'RefVariable' object has no attribute '_id'

# Main Training

In [ ]:
final_information = []

for model_parameters in all_model_parameters:
    
    print("Model parameters chosen:", model_parameters)
    
    X_s, y_1_s, y_2_s, f_s, weights_s = [X, y_1, y_2, f, weights]
        
    f00_s = f_s[:, 0:1]
    f01_s = f_s[:, 1:2]
    f10_s = f_s[:, 2:3]
    f11_s = f_s[:, 3:4]
    
    all_accuracies = {
        "traintotal": 0,
        "train0": 0,
        "train1": 0,
        "train2": 0,
        "train3": 0,
        "testtotal": 0,
        "test0": 0,
        "test1": 0,
        "test2": 0,
        "test3": 0,
    }
    all_losses = {
        "traintotal": 0,
        "train0": 0,
        "train1": 0,
        "train2": 0,
        "train3": 0,
        "testtotal": 0,
        "test0": 0,
        "test1": 0,
        "test2": 0,
        "test3": 0,
    }
    
    for k in range(p["k"]):
        
        # Reset graph
        tf.keras.backend.clear_session()

        print("K:", k)
        
        # Cut out part of kth fold
        lower, upper = bounds[k]
        get_kth = lambda x: x[lower:upper]
        cut_kth = lambda x: np.concatenate([x[:lower], x[upper:samples]], axis=0)
        X_kth, f_kth, weights_kth = list(map(get_kth, [X_s, f_s, weights_s]))
        X_rest, f_rest, weights_rest = list(map(cut_kth, [X_s, f_s, weights_s]))
        
        if model_parameters["resample_data"]:
            resampled_points_per_category = model_parameters["resampled_points_per_category"]

            indexes00 = np.where(f_rest[:, 0] == 1)[0]
            indexes01 = np.where(f_rest[:, 1] == 1)[0]
            indexes10 = np.where(f_rest[:, 2] == 1)[0]
            indexes11 = np.where(f_rest[:, 3] == 1)[0]
            chosen00 = indexes00[np.random.randint(0, indexes00.shape[0], size=resampled_points_per_category)]
            chosen01 = indexes01[np.random.randint(0, indexes01.shape[0], size=resampled_points_per_category)]
            chosen10 = indexes10[np.random.randint(0, indexes10.shape[0], size=resampled_points_per_category)]
            chosen11 = indexes11[np.random.randint(0, indexes11.shape[0], size=resampled_points_per_category)]

            X_train = np.concatenate([
                X_rest[chosen00],
                X_rest[chosen01],
                X_rest[chosen10],
                X_rest[chosen11]
            ], axis=0)
            f_train = np.concatenate([
                f_rest[chosen00],
                f_rest[chosen01],
                f_rest[chosen10],
                f_rest[chosen11]
            ], axis=0)
            weights_train = np.concatenate([
                weights_rest[chosen00],
                weights_rest[chosen01],
                weights_rest[chosen10],
                weights_rest[chosen11]
            ], axis=0)
        else:
            X_train, f_train, weights_train = [X_rest, f_rest, weights_rest]


        # Reset graph
        tf.keras.backend.clear_session()

        model = FhatModel(model_parameters)

        inp_train = dict_to_tensors({"X": X_train, "f": f_train, "weights": weights_train})
        _ = model.train(inp_train)
        final_output = model.run(inp_train, training=False)

        print("Final loss on training data (could be resampled) is", final_output["loss"].numpy())
        print("Final accuracy on training_data (could be resampled) is", final_output["accuracy"].numpy())

        inp_rest = dict_to_tensors({"X": X_rest, "f": f_rest, "weights": weights_rest})
        output_rest = model.run(inp_rest, training=False)

        print("Final accuracy on 4/5 data (not resampled) is", output_rest["accuracy"].numpy())

        inp_kth = dict_to_tensors({"X": X_kth, "f": f_kth, "weights": weights_kth})
        output_kth = model.run(inp_kth, training=False)

        print("Final accuracy on kth fold (not resampled, we never resample this) is", output_kth["accuracy"].numpy())

        def run_all_either_train_or_test_set(X, f, weights, name):
            
            for fhat_category in range(4):
                index = f[:, fhat_category] == 1
                inp = dict_to_tensors({"X": X[index], "f": f[index], "weights": weights[index]})
                output = model.run(inp, training=False)
            
                all_accuracies[name+str(fhat_category)] = \
                    all_accuracies[name+str(fhat_category)] + output["accuracy"].numpy()
                all_losses[name+str(fhat_category)] = \
                    all_losses[name+str(fhat_category)] + output["loss"].numpy()
                
            # Total
            inp = dict_to_tensors({"X": X, "f": f, "weights": weights})
            output = model.run(inp, training=False)
            
            all_accuracies[name+"total"] = \
                all_accuracies[name+"total"] + output["accuracy"].numpy()
            all_losses[name+"total"] = \
                all_losses[name+"total"] + output["loss"].numpy()
            
        run_all_either_train_or_test_set(X_rest, f_rest, weights_rest, "train")
        run_all_either_train_or_test_set(X_kth, f_kth, weights_kth, "test")
        
    
    all_accuracies = {name: value/p["k"] for name, value in all_accuracies.items()}
    all_losses = {name: value/p["k"] for name, value in all_losses.items()}
    final_information.append({"parameters": model_parameters, "all_accuracies": all_accuracies, \
                              "all_losses": all_losses})

In [ ]:
np.save(OUTPUT_FOLDER+PREDICTING_FHAT_ACCURACIES_DATA_PATH, final_information)

In [ ]:
#%%javascript
#IPython.notebook.save_notebook()

In [ ]:
# convert this notebook to html and save it in the folder
#FULL_NOTEBOOK_PATH = OUTPUT_FOLDER+PREDICTING_FHAT_NOTEBOOK_HTML_PATH
#! jupyter nbconvert --to html Predicting_fhat.ipynb  
#! mv Predicting_fhat.html $FULL_NOTEBOOK_PATH